<a href="https://colab.research.google.com/github/akanupam/ML_course/blob/main/week3_ml_zoomcamp_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mutual_info_score,accuracy_score

In [ ]:
link = 'https://raw.githubusercontent.com/akanupam/Datasets/refs/heads/main/course_lead_scoring.csv'

In [ ]:
df = pd.read_csv(link)

In [ ]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [ ]:
df.isnull().sum()

,0
lead_source,128
industry,134
number_of_courses_viewed,0
annual_income,181
employment_status,100
location,63
interaction_count,0
lead_score,0
converted,0


In [ ]:
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [ ]:
categorical = [
    'lead_source','industry','employment_status','location'
]

In [ ]:
numerical = [
    'number_of_courses_viewed', 'annual_income','interaction_count', 'lead_score'
]

In [ ]:
df_categorical = df[categorical]
df_categorical.head()

,lead_source,industry,employment_status,location
0,paid_ads,NaN,unemployed,south_america
1,social_media,retail,employed,south_america
2,events,healthcare,unemployed,australia
3,paid_ads,retail,NaN,australia
4,referral,education,self_employed,europe


In [ ]:
df_numerical = df[numerical]
df_numerical

,number_of_courses_viewed,annual_income,interaction_count,lead_score
0,1,79450.0,4,0.94
1,1,46992.0,1,0.80
2,5,78796.0,3,0.69
3,2,83843.0,1,0.87
4,3,85012.0,3,0.62
...,...,...,...,...
1457,1,NaN,4,0.53
1458,3,65259.0,2,0.24
1459,1,45688.0,3,0.02
1460,5,71016.0,0,0.25


In [ ]:
df_categorical.fillna('NA')
df_numerical.fillna(0)

,number_of_courses_viewed,annual_income,interaction_count,lead_score
0,1,79450.0,4,0.94
1,1,46992.0,1,0.80
2,5,78796.0,3,0.69
3,2,83843.0,1,0.87
4,3,85012.0,3,0.62
...,...,...,...,...
1457,1,0.0,4,0.53
1458,3,65259.0,2,0.24
1459,1,45688.0,3,0.02
1460,5,71016.0,0,0.25


In [ ]:
df['industry'].mode()

,industry
0,retail


Correlation matric for:<br>
interaction_count and lead_score <br>
number_of_courses_viewed and lead_score<br>
number_of_courses_viewed and interaction_count<br>
annual_income and interaction_count<br>

In [ ]:
df_numerical.corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.031551,-0.023565,-0.004879
annual_income,0.031551,1.000000,0.048618,0.005334
interaction_count,-0.023565,0.048618,1.000000,0.009888
lead_score,-0.004879,0.005334,0.009888,1.000000


splitting data

In [ ]:
from sklearn.model_selection import train_test_split
df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state = 42)
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state = 42)

In [ ]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [ ]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [ ]:
df_mi = df_train[categorical].apply(lambda x: mutual_info_score(x.fillna('NA'), df_train.converted))
df_mi = df_mi.sort_values(ascending = False).to_frame('MI')
display(df_mi)

,MI
lead_source,0.035396
employment_status,0.012938
industry,0.011575
location,0.004464


In [ ]:
del df_train['converted']
del df_val['converted']
del df_test['converted']

Model training


In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [ ]:
df_train[categorical] = df_train[categorical].fillna('NA')
df_train[numerical] = df_train[numerical].fillna(0)
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)
train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)


In [ ]:
model.fit(X_train,y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [ ]:
df_val[categorical] = df_val[categorical].fillna('NA')
df_val[numerical] = df_val[numerical].fillna(0)

val_dict = df_val.to_dict(orient = 'records')
X_val = dv.fit_transform(val_dict)

y_pred = model.predict(X_val)

In [ ]:
acc_score = round(accuracy_score(y_val,y_pred),2)
acc_score

0.7